In [17]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch_geometric.data import Data
from torch_geometric.nn import TransformerConv, TopKPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.loader import DataLoader
import gpytorch
from gpytorch.mlls import DeepApproximateMLL, VariationalELBO
from gpytorch.likelihoods import GaussianLikelihood
import os
import sys
import random
loger_dir = os.path.abspath('/home/ubuntu/project/mayang/LOGER')

# 将 QPE 目录添加到 sys.path
sys.path.append(loger_dir)
from core.models.SpectralDeltaGP import SpectralDeltaGP
from core.models.DGP import DeepGPModel
from core.models.DKL import DKLGPModel
from sql import Sql, _global
from core import database, Sql, Plan, load
import ast
import re

# 获取 QPE 目录的绝对路径
qpe_dir = os.path.abspath('/home/ubuntu/project/mayang')

# 将 QPE 目录添加到 sys.path
sys.path.append(qpe_dir)

# 现在尝试导入 TreeBuilder 类
from QPE.sql2fea import TreeBuilder

def parse_attribute_config(index_str):
    '''处理输入的configuration，转化成cols/attribute的形式'''
    # 将字符串转换为列表
    index_list = ast.literal_eval(index_str)
    
    table_cols_list = []
    for index in index_list:
        # 去掉 I() 的部分，提取表名和列名
        index_content = index[4:-1]  # 去掉前面的 'I(' 和后面的 ')'
        table_cols = index_content.split(',')
        table_cols_list.append(table_cols)
    
    return table_cols_list

def swap_join_condition(condition: str) -> str:
    '''防止同一个连接条件，左右连接条件不一样而无法识别'''
    # 找到等号的位置
    equal_index = condition.find('=')

    # 如果没有找到等号，返回原字符串
    if equal_index == -1:
        return condition

    # 拆分等号两边的条件，并去掉左右空格
    left_side = condition[:equal_index].strip()
    right_side = condition[equal_index + 1:].strip()

    # 交换等式两边的条件
    swapped_condition = f"{right_side} = {left_side}"

    return swapped_condition


def has_text_expression(cmp: str) -> bool:
    '''判断cmp中有没有文本'''
    # 使用正则表达式查找被引号包围的文本
    pattern = r"'[^']*'"

    # 查找匹配的内容
    match = re.search(pattern, cmp)

    # 如果找到了匹配内容，返回 True；否则返回 False
    return match is not None


def convert_between_expression(cmp: str) -> str:
    '''转换 BETWEEN 表达式'''
    match = re.match(r"([\w\.]+)\.(\w+)\s*BETWEEN\s*(.*)\s*AND\s*(.*)", cmp)

    if not match:
        raise ValueError("The input string is not a valid BETWEEN expression.")

    table_name, column, lower_bound, upper_bound = match.groups()

    # 检查值是否是数字
    if lower_bound.strip().isdigit() and upper_bound.strip().isdigit():
        new_expression = f"(({column} >= {lower_bound.strip()}) AND ({column} <= {upper_bound.strip()}))"
    else:
        new_expression = f"(({column})::text >= {lower_bound.strip()}::text) AND (({column})::text <= {upper_bound.strip()}::text)"

    return table_name, new_expression[:-1]


def convert_to_text_comparison(cmp: str):
    '''转化cmp与plan中的谓词表示, 处理 IN 语句。'''
    match_eq = re.match(r"([\w\.]+)\.(\w+)\s*(=|<>|<|>|<=|>=)\s*'(.*)'", cmp)
    match_in = re.match(r"([\w\.]+)\.(\w+)\s*IN\s*\((.*)\)", cmp, re.DOTALL)

    if match_eq:
        # 处理 '=' 比较
        table_name, column, operator, value = match_eq.groups()
        new_expression = f"(({column})::text {operator} '{value}'::text)"
        return table_name, new_expression
    elif match_in:
        # 处理 'IN' 比较
        table_name, column, values = match_in.groups()

        # 使用正则提取引号中的值，允许多行
        value_list = re.findall(r"'(.*?)'", values)
        if not value_list:
            raise ValueError("在 IN 表达式中未找到有效值。")

        # 检查是否存在嵌套括号
        if '(' in values or ')' in values:
            new_expression = f"(({column})::text = ANY ("
            return table_name, new_expression

        # 检查是否有值包含两个单词
        if len(value_list) > 1 and any(len(value.split()) > 1 for value in value_list):
            new_expression = f"(({column})::text = ANY ("
            return table_name, new_expression

        # 检查是否只有一个值
        if len(value_list) == 1:
            new_expression = f"(({column})::text = '{value_list[0]}'::text)"
        else:
            formatted_values = ','.join(value_list)  # 拼接多个值
            new_expression = f"(({column})::text = ANY ('{{{formatted_values}}}'::text[]))"

        return table_name, new_expression

    raise ValueError("输入字符串不是有效的比较表达式。")


def trim_parentheses_and_spaces(input_str: str) -> str:
    # 去除开头的空格
    trimmed_str = input_str.lstrip()
    # 如果开头是 '('，则去除它
    if trimmed_str.startswith('('):
        trimmed_str = trimmed_str[1:].lstrip()  # 去掉 '(' 以及后面的空格
    return trimmed_str


def transform_sql_like_condition(sql_condition):
    '''处理LIKE和NOT LIKE条件'''
    # 匹配 `AND` 和 `OR`
    sql_condition = trim_parentheses_and_spaces(sql_condition)
    conditions = re.split(r"(?i) AND | OR ", sql_condition)
    # print(conditions)

    # 转换每个 LIKE 和 NOT LIKE 条件为 Postgres 的格式
    transformed_clauses = []
    for condition in conditions:
        condition = condition.strip()
        table_name = condition.split('.')[0]
        col_name = condition.split()[0][len(table_name) + 1:]
        if 'not like' in condition.lower():
            transformed_clause = f"(({col_name})::text !~~ {condition.split('not like')[1].strip().rstrip(')')}::text)"
        elif 'like' in condition.lower():
            transformed_clause = f"(({col_name})::text ~~ {condition.split('like')[1].strip().rstrip(')')}::text)"
            # print("condition.split('like')[1].strip()",condition.split('like')[1].strip())
        else:
            continue

        transformed_clauses.append(transformed_clause)

    # 返回表名和第一个转换后的条件
    return table_name, transformed_clauses[0]


def delete_left_table_cond(sql_condition):
    # 使用正则表达式匹配表名和列名
    match = re.match(r"[\w\.]+\.(\w+)\s*=\s*(.*)", sql_condition)

    if not match:
        raise ValueError("The input string is not a valid comparison expression.")

    # 提取列名和右侧表达式
    column = match.group(1)
    right_side = match.group(2)

    return f"{column} = {right_side.strip()}"


def dgl_node_and_edge_vectorization(sql_query, config_index, plan,attribute_dict):
    sql_instance = Sql(sql_query)

    # 构建异构图
    g, data_dict, node_indexes, edge_list = sql_instance.to_hetero_graph_dgl()
    # 构建 node feature
    filter_features = g.ndata['filter']
    edge_features = g.ndata['edge']
    onehot_features = g.ndata['onehot']
    others_features = g.ndata['others'].view(g.ndata['filter'].shape[0], -1)

    # 假设两个特征的维度是相同的
    combined_features = torch.cat((filter_features, edge_features, onehot_features, others_features), dim=1)
    g.ndata['feature'] = combined_features

    # 构建 edge feature
    tree_builder = TreeBuilder()
    tree_builder.set_configruations(config_index)
    tree_builder.set_table_to_alias_dict(sql_query)
    # 准备您的 SQL 执行计划数据，这里是一个示例
    execution_plan = ast.literal_eval(plan)[0]['Plan']

    # 特征化执行计划
    features = tree_builder.plan_to_feature_tree(execution_plan, current_height=0)

    # 提取对应operator vector
    operator_vector_dict = tree_builder.get_operator_vector()
    # 建立cmp与plan_operator的映射关系
    tables = []
    operator_vectors_cmp1 = []
    operator_vectors_cmp2 = []
    get_keys = []
    for cmp, table_num in edge_list:
        flag = False
        # print('cmp:',cmp)
        if table_num == 1:
            if 'like' in cmp and has_text_expression(cmp):
                try:
                    table, cmp_in_op = transform_sql_like_condition(cmp)
                except:
                    print('error process like_cmp:', cmp)
            elif 'BETWEEN' in cmp:
                table, cmp_in_op = convert_between_expression(cmp)
            elif has_text_expression(cmp):
                table, cmp_in_op = convert_to_text_comparison(cmp)
            else:
                table, cmp_in_op = cmp.split('.')

        for key in operator_vector_dict:
            if table_num == 1:
                table_op, key_op = key.split('_47_')
                if cmp_in_op in key and table == table_op:
                    tables.append(table)
                    operator_vectors_cmp1.append(operator_vector_dict[key])
                    flag = True
                    break
            # print(key)
            elif cmp in key or swap_join_condition(cmp) in key or delete_left_table_cond(
                    cmp) in key or delete_left_table_cond(swap_join_condition(cmp)) in key:
                get_keys.append(cmp)
                operator_vectors_cmp2.append(operator_vector_dict[key])
                operator_vectors_cmp2.append(operator_vector_dict[key])
                flag = True
                break
        if not flag:
            # 存在plan中不存在的cmp
            print('error cmp', cmp)
            if table_num == 2:
                print('add edge vector:', table_num)
                operator_vectors_cmp2.append(torch.zeros((1, num_edge_features)))
                operator_vectors_cmp2.append(torch.zeros((1, num_edge_features)))
    # print('cmp',len(edge_list))
    # print(len(operator_vectors_cmp1),len(operator_vectors_cmp2))
    # 合并每一条边的向量
    vector_cmp1 = torch.cat(operator_vectors_cmp1, dim=0)
    vector_cmp2 = torch.cat(operator_vectors_cmp2, dim=0)
    vector_edge = torch.cat((vector_cmp2, vector_cmp1), dim=0)
    # 添加自环
    for t in tables:
        new_u = torch.tensor([node_indexes['~' + t]])  # 自环的起点
        new_v = torch.tensor([node_indexes['~' + t]])  # 自环的终点
        g.add_edges(new_u, new_v)
    g.edata['feature'] = vector_edge
    
    #构建configuration特征
    configuration_vector=[0]*len(attribute_dict)
    for attrs in parse_attribute_config(config_index):
        for p in range(len(attrs)):
            #获得在indexed_attribute中的位置
            attr=attrs[p]
            if attr[:2]=='C ':
                attr=attr[2:]
            attr_p=attribute_dict[attr]
            configuration_vector[attr_p]+=1/(p+1)
    g.global_data = {"configuration_vector": torch.tensor(configuration_vector)}
    return g


def dgl_to_pyg(dgl_graph,label):
    """将 DGL 图转换为 PyG 格式."""
    x = dgl_graph.ndata['feature']  # 节点特征
    edge_index = torch.stack(dgl_graph.edges()).long()  # 边索引 (转换为 PyG 格式)
    edge_attr = dgl_graph.edata['feature']  # 边特征
    configuration_vector = dgl_graph.global_data['configuration_vector'] # configuration_vector
    batch = torch.zeros(dgl_graph.number_of_nodes(), dtype=torch.long)  # 所有节点归为同一批次
    label_tensor = torch.tensor(label, dtype=torch.float32)  # 根据需要使用 float32 或其他类型
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, configuration_vector=configuration_vector,y=label_tensor, batch=batch)


# class GraphEncoder(nn.Module):
#     """图编码模块，基于 TransformerConv."""

#     def __init__(self, in_feats, edge_feats, embedding_size, num_layers=3, top_k_every_n=3, top_k_ratio=0.5, n_heads=4,
#                  dropout_rate=0.5):
#         super(GraphEncoder, self).__init__()
#         self.embedding_size = embedding_size
#         self.top_k_every_n = top_k_every_n
#         self.edge_dim = edge_feats
#         self.n_heads = n_heads

#         # 第一层 TransformerConv
#         self.conv1 = TransformerConv(in_feats, self.embedding_size, heads=n_heads, dropout=dropout_rate,
#                                      edge_dim=edge_feats, beta=True)
#         self.transf1 = nn.Linear(self.embedding_size * n_heads, self.embedding_size)
#         self.bn1 = nn.BatchNorm1d(self.embedding_size)

#         # 其他 TransformerConv 层及 TopKPooling
#         # TopKPooling 需要对图进行下采样，类似于卷积神经网络（CNN）中的池化操作，用来减小图的大小，保留最重要的信息。
#         self.conv_layers = nn.ModuleList()
#         self.transf_layers = nn.ModuleList()
#         self.bn_layers = nn.ModuleList()
#         self.pooling_layers = nn.ModuleList()

#         for _ in range(num_layers):
#             self.conv_layers.append(
#                 TransformerConv(self.embedding_size, self.embedding_size, heads=n_heads, dropout=dropout_rate,
#                                 edge_dim=edge_feats, beta=True))
#             self.transf_layers.append(nn.Linear(self.embedding_size * n_heads, self.embedding_size))
#             self.bn_layers.append(nn.BatchNorm1d(self.embedding_size))
#             self.pooling_layers.append(TopKPooling(self.embedding_size, ratio=top_k_ratio))

#     def forward(self, x, edge_attr, edge_index, batch_index):
#         # print('x',x.shape)
#         x = self.conv1(x, edge_index, edge_attr)

#         # 调整线性层输入的形状
#         x = torch.relu(self.transf1(x.view(-1, self.embedding_size * self.n_heads)))
#         x = self.bn1(x)

#         global_representation = []

#         for i, conv_layer in enumerate(self.conv_layers):
#             x = conv_layer(x, edge_index, edge_attr)
#             x = torch.relu(self.transf_layers[i](x.view(-1, self.embedding_size * self.n_heads)))  # 确保形状匹配
#             x = self.bn_layers[i](x)

#             if i % self.top_k_every_n == 0 or i == len(self.conv_layers) - 1:
#                 # print('edge_index', edge_index.shape)
#                 x, edge_index, edge_attr, batch_index, _, _ = self.pooling_layers[i](
#                     x, edge_index, edge_attr, batch_index
#                 )
#                 global_representation.append(torch.cat([
#                     gmp(x, batch_index),
#                     gap(x, batch_index)
#                 ], dim=1))

#         x = sum(global_representation)

#         return x

class GraphEncoder(nn.Module):
    """图编码模块，基于 TransformerConv,添加了残差连接。"""

    def __init__(self, in_feats, edge_feats, embedding_size, num_layers=3, top_k_every_n=3, top_k_ratio=0.5, n_heads=4,
                 dropout_rate=0.5):
        super(GraphEncoder, self).__init__()
        self.embedding_size = embedding_size
        self.top_k_every_n = top_k_every_n
        self.edge_dim = edge_feats
        self.n_heads = n_heads

        # 第一层 TransformerConv
        self.conv1 = TransformerConv(in_feats, self.embedding_size, heads=n_heads, dropout=dropout_rate,
                                     edge_dim=edge_feats, beta=True)
        self.transf1 = nn.Linear(self.embedding_size * n_heads, self.embedding_size)
        self.bn1 = nn.BatchNorm1d(self.embedding_size)

        # 其他 TransformerConv 层及 TopKPooling
        self.conv_layers = nn.ModuleList()
        self.transf_layers = nn.ModuleList()
        self.bn_layers = nn.ModuleList()
        self.pooling_layers = nn.ModuleList()

        for _ in range(num_layers):
            self.conv_layers.append(
                TransformerConv(self.embedding_size, self.embedding_size, heads=n_heads, dropout=dropout_rate,
                                edge_dim=edge_feats, beta=True))
            self.transf_layers.append(nn.Linear(self.embedding_size * n_heads, self.embedding_size))
            self.bn_layers.append(nn.BatchNorm1d(self.embedding_size))
            self.pooling_layers.append(TopKPooling(self.embedding_size, ratio=top_k_ratio))

    def forward(self, x, edge_attr, edge_index, batch_index):
        # 残差连接的初始输入
        residual = x

        # 第一个 TransformerConv 层及线性变换
        x = self.conv1(x, edge_index, edge_attr)
        x = torch.relu(self.transf1(x.view(-1, self.embedding_size * self.n_heads)))
        x = self.bn1(x)

        # 添加残差连接，将初始输入加入 x
        if residual.shape == x.shape:
            x = x + residual  # 进行残差连接

        global_representation = []

        for i, conv_layer in enumerate(self.conv_layers):
            # 每一层的残差输入
            residual = x

            # TransformerConv 和线性变换
            x = conv_layer(x, edge_index, edge_attr)
            x = torch.relu(self.transf_layers[i](x.view(-1, self.embedding_size * self.n_heads)))  # 确保形状匹配
            x = self.bn_layers[i](x)

            # 残差连接
            if residual.shape == x.shape:
                x = x + residual  # 进行残差连接

            if i % self.top_k_every_n == 0 or i == len(self.conv_layers) - 1:
                x, edge_index, edge_attr, batch_index, _, _ = self.pooling_layers[i](
                    x, edge_index, edge_attr, batch_index
                )
                global_representation.append(torch.cat([
                    gmp(x, batch_index),
                    gap(x, batch_index)
                ], dim=1))

        # 将全局表示相加作为最终输出
        x = sum(global_representation)

        return x



class ImprovementPredictionModel(nn.Module):
    """预测模型，基于图编码，并拼接额外的特征，支持 Spectral Delta GP."""

    def __init__(self, in_feats, edge_feats, graph_embedding_size, config_vector_size, use_dgp=True, use_spectral_delta_gp=False):
        """
        :param in_feats: 输入特征的维度
        :param edge_feats: 边的特征维度
        :param graph_embedding_size: 图嵌入维度
        :param config_vector_size: 配置向量维度
        :param use_dgp: 是否使用深度高斯过程模型
        :param use_spectral_delta_gp: 是否使用 Spectral Delta GP 模型
        :param num_deltas: Spectral Delta GP 中的 delta 数量
        """
        print('start ImprovementPredictionModel')
        super(ImprovementPredictionModel, self).__init__()
        self.use_dgp = use_dgp
        self.use_spectral_delta_gp = use_spectral_delta_gp
        self.graph_encoder = GraphEncoder(in_feats, edge_feats, graph_embedding_size)

        # 拼接后的特征维度
        self.input_size = graph_embedding_size * 2 + config_vector_size

        if self.use_spectral_delta_gp:
            # 使用 Spectral Delta GP
            print('using Spectral Delta GP')
            self.gp_model = SpectralDeltaGP(input_dims=self.input_size, num_deltas=1)
            self.likelihood = self.gp_model.likelihood  # 使用 Spectral Delta GP 的似然函数
        elif self.use_dgp:
            # 使用深度高斯过程
            print('using DeepGPModel')
            self.likelihood = GaussianLikelihood()
            self.gp_model = DeepGPModel(self.input_size)
        else:
            # 使用 DKL GP 模型
            print('using DKLGPModel')
            self.likelihood = GaussianLikelihood()
            self.gp_model = DKLGPModel(self.input_size)

    def forward(self, x, edge_attr, edge_index, configuration_vector, batch_index, target):
        # 获取图嵌入
        graph_embedding = self.graph_encoder(x, edge_attr, edge_index, batch_index)

        #TODO 修改batch需要修改32
        configuration_vector = configuration_vector.view(32, -1)

        # 拼接额外的特征
        graph_embedding = torch.cat([graph_embedding, configuration_vector], dim=1)
        
        # 设置训练数据
        self.gp_model.set_train_data(inputs=graph_embedding, targets=target, strict=False)


        # 确保在训练时调用 train()，在预测时调用 eval()
        self.gp_model.train()
        self.likelihood.train()

        with gpytorch.settings.fast_pred_var():
            # 使用拼接后的 embedding 进行预测
            output = self.gp_model(graph_embedding)
            prediction = self.likelihood(output)

        return prediction  # 返回高斯过程的预测分布

    def predict(self, x, edge_attr, edge_index, configuration_vector, batch_index):
        """在预测时调用"""
        graph_embedding = self.graph_encoder(x, edge_attr, edge_index, batch_index)
        configuration_vector = configuration_vector.view(1, -1)

        # 拼接额外的特征
        graph_embedding = torch.cat([graph_embedding, configuration_vector], dim=1)

        self.gp_model.eval()
        self.likelihood.eval()

        with gpytorch.settings.fast_pred_var():
            output = self.gp_model(graph_embedding)
            prediction = self.likelihood(output)

        return prediction




def load_sql_graphs_pyg(csv_path, dbname, user, password, host, port, start_idx=0, end_idx=8877):
    """
    从指定的 CSV 文件加载 SQL 查询，并将它们转换为 PyG 图数据。

    参数:
    - csv_path (str): CSV 文件路径。
    - dbname (str): 数据库名称。
    - user (str): 数据库用户。
    - password (str): 数据库密码。
    - host (str): 数据库主机地址。
    - port (str): 数据库端口。
    - start_idx (int): SQL 查询的起始索引。
    - end_idx (int): SQL 查询的结束索引。

    返回:
    - sql_graphs_pyg (list): PyG 数据列表。
    """
    print('start load_sql_graphs_pyg')
    # 读取 CSV 数据
    df = pd.read_csv(csv_path).iloc[start_idx:, :]
    # df = pd.read_csv(csv_path).iloc[start_idx:, :].sample(n=3200, random_state=42)
    # 设置数据库连接
    database.setup(dbname=dbname, user=user, password=password, host=host, port=port, cache=False)
    sql_graphs_pyg = []
    #创建index的vector
    attribute_num=0
    attribute_dict={}
    for x in df['index'].values:
        for attrs in parse_attribute_config(x):
            for attr in attrs:
                if attr[:2]=='C ':
                    attr=attr[2:]
                if attr not in attribute_dict:
                    attribute_dict[attr]=attribute_num
                    attribute_num+=1


    query_list = df['query'].values
    quey_config_list = df['index'].values
    query_plans = df['query_plan_no_index'].values
    for i in range(df.shape[0]):
        # if cnt % 50 == 0:
        #     print(cnt)
        print(i)
        g = dgl_node_and_edge_vectorization(query_list[i], quey_config_list[i], query_plans[i], attribute_dict)
        # 转换为 PyG 数据
        pyg_data = dgl_to_pyg(g,df['label'].values[i])
        sql_graphs_pyg.append(pyg_data)

    return sql_graphs_pyg, df['label'].values, len(attribute_dict)


num_edge_features = 15

    # 读取数据并设置数据库
sql_graphs_pyg, label_list, config_vector_size = load_sql_graphs_pyg(
        csv_path='/home/ubuntu/project/mayang/Classification/process_data/job/job_train_8935.csv',
        dbname='imdbload',
        user='postgres',
        password='password',
        host='127.0.0.1',
        port='5432',
        start_idx=7
    )
print('len(label_list) 需要是batch_size的倍数', len(label_list))
# 设置随机种子
seed = 42
torch.manual_seed(seed)  # PyTorch 随机种子
np.random.seed(seed)  # NumPy 随机种子
random.seed(seed)  # Python 随机种子
    # 创建 PyG DataLoader
data_loader = DataLoader(sql_graphs_pyg, batch_size=32, shuffle=True)

start load_sql_graphs_pyg
{'dbname': 'imdbload', 'user': 'postgres', 'password': 'password', 'host': '127.0.0.1', 'port': '5432', 'cache': False}
<connection object at 0x7fe04d90c180; dsn: 'user=postgres password=xxx dbname=imdbload host=127.0.0.1 port=5432', closed: 0>
<cursor object at 0x7fe04d90d7c0; closed: 0>
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
error cmp ci.movie_id = mc.movie_id
add edge vector: 2
57
error cmp ci.movie_id = mc.movie_id
add edge vector: 2
58
error cmp ci.movie_id = mc.movie_id
add edge vector: 2
59
error cmp ci.movie_id = mc.movie_id
add edge vector: 2
60
error cmp ci.movie_id = mc.movie_id
add edge vector: 2
61
error cmp ci.movie_id = mc.movie_id
add edge vector: 2
62
error cmp ci.movie_id = mc.movie_id
add edge vector: 2
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
1

In [40]:
sql_graphs_pyg[0]

Data(x=[9, 201], edge_index=[2, 29], edge_attr=[29, 15], y=0.7762723565101624, configuration_vector=[44], batch=[9])

In [41]:
for i in sql_graphs_pyg[0]['edge_attr']:
    print(i)

tensor([0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.4700, 1.0000, 1.0000,
        8.0000, 0.4700, 0.0000, 1.0000, 1.0000, 1.0000])
tensor([0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.4700, 1.0000, 1.0000,
        8.0000, 0.4700, 0.0000, 1.0000, 1.0000, 1.0000])
tensor([0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        2.4065e+05, 5.4704e+04, 8.8496e-03, 9.0000e+00, 2.3852e+05, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00])
tensor([0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        2.4065e+05, 5.4704e+04, 8.8496e-03, 9.0000e+00, 2.3852e+05, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00])
tensor([0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.1800, 1.0000, 1.0000,
        2.0000, 0.1800, 0.0000, 1.0000, 1.0000, 1.0000])
tensor([0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.1800, 1.0000, 1.0000,
        2.0000, 0.1800, 0.0000, 1.0000, 1.0000, 1.0000])
tensor([0.0000e+00, 1.0000e+00, 0.0000e+00

In [44]:
class ImprovementPredictionModel(nn.Module):
    """预测模型，基于图编码，并拼接额外的特征，支持 Spectral Delta GP."""

    def __init__(self, in_feats, edge_feats, graph_embedding_size, config_vector_size, use_dgp=True, use_spectral_delta_gp=False):
        """
        :param in_feats: 输入特征的维度
        :param edge_feats: 边的特征维度
        :param graph_embedding_size: 图嵌入维度
        :param config_vector_size: 配置向量维度
        :param use_dgp: 是否使用深度高斯过程模型
        :param use_spectral_delta_gp: 是否使用 Spectral Delta GP 模型
        :param num_deltas: Spectral Delta GP 中的 delta 数量
        """
        print('start ImprovementPredictionModel')
        super(ImprovementPredictionModel, self).__init__()
        self.use_dgp = use_dgp
        self.use_spectral_delta_gp = use_spectral_delta_gp
        self.graph_encoder = GraphEncoder(in_feats, edge_feats, graph_embedding_size)

        # 拼接后的特征维度
        self.input_size = graph_embedding_size * 2 + config_vector_size
        self.normalizer = torch.nn.BatchNorm1d(self.input_size)

        if self.use_spectral_delta_gp:
            # 使用 Spectral Delta GP
            print('using Spectral Delta GP')
            self.gp_model = SpectralDeltaGP(input_dims=self.input_size, num_deltas=100)
            self.likelihood = self.gp_model.likelihood  # 使用 Spectral Delta GP 的似然函数
        elif self.use_dgp:
            # 使用深度高斯过程
            print('using DeepGPModel')
            self.likelihood = GaussianLikelihood()
            self.gp_model = DeepGPModel(self.input_size)
        else:
            # 使用 DKL GP 模型
            print('using DKLGPModel')
            self.likelihood = GaussianLikelihood()
            self.gp_model = DKLGPModel(self.input_size)

    def forward(self, x, edge_attr, edge_index, configuration_vector, batch_index, target):
        # 获取图嵌入
        graph_embedding = self.graph_encoder(x, edge_attr, edge_index, batch_index)

        #TODO 修改batch需要修改32
        configuration_vector = configuration_vector.view(32, -1)

        # 拼接额外的特征
        graph_embedding = torch.cat([graph_embedding, configuration_vector], dim=1)
        
        # 应用标准化
        graph_embedding = self.normalizer(graph_embedding)
        
        # 设置训练数据
        self.gp_model.set_train_data(inputs=graph_embedding, targets=target, strict=False)


        # 确保在训练时调用 train()，在预测时调用 eval()
        self.gp_model.train()
        self.likelihood.train()

        with gpytorch.settings.fast_pred_var():
            # 使用拼接后的 embedding 进行预测
            output = self.gp_model(graph_embedding)
            prediction = self.likelihood(output)

        return prediction  # 返回高斯过程的预测分布

    def predict(self, x, edge_attr, edge_index, configuration_vector, batch_index):
        """在预测时调用"""
        graph_embedding = self.graph_encoder(x, edge_attr, edge_index, batch_index)
        configuration_vector = configuration_vector.view(1, -1)

        # 拼接额外的特征
        graph_embedding = torch.cat([graph_embedding, configuration_vector], dim=1)
        
        # 应用标准化
        graph_embedding = self.normalizer(graph_embedding)

        self.gp_model.eval()
        self.likelihood.eval()

        with gpytorch.settings.fast_pred_var():
            output = self.gp_model(graph_embedding)
            prediction = self.likelihood(output)

        return prediction

In [45]:
model = ImprovementPredictionModel(sql_graphs_pyg[0].x.shape[1], sql_graphs_pyg[0].edge_attr.shape[1],
                                       graph_embedding_size=32,config_vector_size=config_vector_size,use_spectral_delta_gp=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

print('start DeepApproximateMLL')
mll = gpytorch.mlls.ExactMarginalLogLikelihood(model.gp_model.likelihood, model.gp_model)
total_loss_list=[float('inf')]
for epoch in range(30):
    model.train()
    total_loss = 0
    for batch_idx, batch in enumerate(data_loader):
        # 前向传播
        pred_time = model(batch.x, batch.edge_attr, batch.edge_index, batch.configuration_vector, batch.batch,batch.y)

        target = batch.y.view(-1)
        loss = -mll(pred_time, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        print("Epoch %d batch_idx%d loss:" % (epoch + 1, batch_idx), loss.item())

    print(f'Epoch {epoch + 1}, Loss: {total_loss:.4f}')

    # 随机抽取 20 个样本进行预测
    sample_indices = random.sample(range(len(sql_graphs_pyg)), 50)
    min_diff=[]
    for idx in sample_indices:
        sample_batch = sql_graphs_pyg[idx]
        sample_x = sample_batch.x
        sample_edge_attr = sample_batch.edge_attr
        sample_edge_index = sample_batch.edge_index
        sample_configuration_vector= sample_batch.configuration_vector

        # 获取对应的 target 值
        target_value = label_list[idx]
        model.eval()
        with torch.no_grad():
            pred_mean = model.predict(sample_x, sample_edge_attr, sample_edge_index, sample_configuration_vector, sample_batch.batch)
        diff=np.mean(pred_mean.mean.numpy())-target_value
        print(
            f'Sample Index: {idx}, Predicted Mean: {np.mean(pred_mean.mean.numpy())}, Target Value: {target_value} difference: {diff} Variance: {pred_mean.variance.numpy()}')
        if abs(diff)<0.1:
            min_diff.append(diff)
    print('the number of samples < 0.1:',len(min_diff))
    #一个Epoch提升的loss小于1就退出
    if total_loss>total_loss_list[-1]-1:
            break
    total_loss_list.append(total_loss)

start ImprovementPredictionModel
using Spectral Delta GP
start DeepApproximateMLL
Epoch 1 batch_idx0 loss: 1.0629689693450928
Epoch 1 batch_idx1 loss: 1.0036183595657349
Epoch 1 batch_idx2 loss: 0.9323734045028687
Epoch 1 batch_idx3 loss: 0.9536679983139038
Epoch 1 batch_idx4 loss: 0.930885374546051
Epoch 1 batch_idx5 loss: 0.8809640407562256
Epoch 1 batch_idx6 loss: 0.9203877449035645
Epoch 1 batch_idx7 loss: 0.9591999650001526
Epoch 1 batch_idx8 loss: 0.9842515587806702
Epoch 1 batch_idx9 loss: 0.8061074614524841
Epoch 1 batch_idx10 loss: 0.8928434252738953
Epoch 1 batch_idx11 loss: 0.9486167430877686
Epoch 1 batch_idx12 loss: 0.8575764298439026
Epoch 1 batch_idx13 loss: 0.8827961683273315
Epoch 1 batch_idx14 loss: 0.9244781732559204
Epoch 1 batch_idx15 loss: 0.8345780372619629
Epoch 1 batch_idx16 loss: 0.8679741024971008
Epoch 1 batch_idx17 loss: 0.840520977973938
Epoch 1 batch_idx18 loss: 0.7986804842948914
Epoch 1 batch_idx19 loss: 0.8156138062477112
Epoch 1 batch_idx20 loss: 0.89

In [46]:
diff_list=[]
for idx in range(8928):
    sample_batch = sql_graphs_pyg[idx]
    sample_x = sample_batch.x
    sample_edge_attr = sample_batch.edge_attr
    sample_edge_index = sample_batch.edge_index
    sample_configuration_vector= sample_batch.configuration_vector

    # 获取对应的 target 值
    target_value = label_list[idx]
    model.eval()
    with torch.no_grad():
        with gpytorch.settings.num_likelihood_samples(32):
            pred_mean = model.predict(sample_x, sample_edge_attr, sample_edge_index, sample_configuration_vector, sample_batch.batch)
    print(
            f'Sample Index: {idx}, Predicted Mean: {np.mean(pred_mean.mean.numpy())}, Target Value: {target_value}, difference: {np.mean(pred_mean.mean.numpy())-target_value} Variance: {pred_mean.variance.numpy()}')
    diff_list.append(np.mean(pred_mean.mean.numpy())-target_value)

Sample Index: 0, Predicted Mean: 0.25668054819107056, Target Value: 0.7762723471220767, difference: -0.5195917989310062 Variance: [0.10798024]
Sample Index: 1, Predicted Mean: 0.21416588127613068, Target Value: 0.8062924517534559, difference: -0.5921265704773252 Variance: [0.11761013]
Sample Index: 2, Predicted Mean: 0.479180246591568, Target Value: 0.7784686092591014, difference: -0.29928836266753345 Variance: [0.09753381]
Sample Index: 3, Predicted Mean: 0.009935885667800903, Target Value: 0.7749570081760407, difference: -0.7650211225082398 Variance: [0.10676001]
Sample Index: 4, Predicted Mean: 0.3965036869049072, Target Value: 0.7777587391149067, difference: -0.38125505220999945 Variance: [0.1177488]
Sample Index: 5, Predicted Mean: 0.5767021179199219, Target Value: 0.8174958364803653, difference: -0.2407937185604434 Variance: [0.11762336]
Sample Index: 6, Predicted Mean: 0.28604331612586975, Target Value: 0.8145510698040941, difference: -0.5285077536782243 Variance: [0.11054334]
S

In [47]:
less_1=[x for x in diff_list if abs(x)<0.1]
len(less_1)

1292

In [22]:
pred_mean.variance

tensor([0.0553])